In [43]:
import nibabel as nib
import glob
import sys
sys.path.append('gradient_data/src/')
import os
import scipy.stats
import numpy as np
import statsmodels.sandbox.stats.multicomp
import nibabel as nib

In [ ]:
### Comparison of cerebello-cerebral connectivity between cerebellar Gradient 1 and 2 peaks at each area of
### motor and nonmotor representation was calculated as follows:
### 1) Using workbench view, save dscalar map of connectivity from peak of each Gradient (e.g. "L2midcog_fconn1003subjects.dscalar.nii")
### 2) Using workbench view, annotate correlation values between seeds:
###                           Fisher_z
### R1mot R2mot               0.231264
### L1mot L2mot               0.23966
### R12highcog R3highcog      0.313801
### L12highcog L3highcog      0.370525
### R1midcog R2midcog         0.450554
### L1midcog L2midcog         0.330115
### R1midcog R3midcog         0.330242
### L1midcog L3midcog         0.222103
### R1midcog R3midcog_alt     0.245531
### L1midcog L3midcog_alt     0.336851
### R2midcog R3midcog         0.276653
### L2midcog L3midcog         0.154236
### R2midcog R3midcog_alt     0.188767
### L2midcog L3midcog_alt     0.218249
### "Alt" refers to peak of Gradient 2 in lobule IX if peak was in lobule X, and vice versa. These values were
### not included in the final analysis
### 3) Contrast r values between seeds using the equations from Meng et al., 1992:
### wb_command -cifti-math '((var_zr1)-(var_zr2))*((sqrt(1003-3))/(2*(1-0.301)*((1-(((1-0.301)/(2*(1-(((tanh(var_zr1)^2)+(tanh(var_zr2)^2))/2)))+((1-0.301)/(2*(1-((( tanh(var_zr1)^2)+( tanh(var_zr2)^2))/2)))>1)*(1-(1-0.301)/(2*(1-(((tanh(var_zr1)^2)+(tanh(var_zr2)^2))/2))))))*((( tanh(var_zr1)^2)+( tanh(var_zr2)^2))/2))/(1-(((tanh(var_zr1)^2)+(tanh(var_zr2)^2))/2)))))' comparison_L2midcog_vs_L1midcog_1003subjects.dscalar.nii -var var_zr1 L2midcog_fconn1003subjects.dscalar.nii -var var_zr2 L1midcog_fconn1003subjects.dscalar.nii
### Substitute 0.301 with each particular correlation value between seeds. Note that z values have to be converted to r values (e.g. L1midcog L2midcog z=0.330115 corresponds to r=0.301)
### 4) Correct comparison maps with FDR as follows:

In [36]:
directory = 'xxx' ### write directory with comparison maps here
files = os.listdir(directory)

In [52]:
### Check that files are loaded
for x in files:
    if x.endswith('motor.dscalar.nii'):
        print(x)

comparison_L2midnonmotor_vs_L1midnonmotor.dscalar.nii
comparison_L2motor_vs_L1motor.dscalar.nii
comparison_L3highnonmotor_vs_L12highnonmotor.dscalar.nii
comparison_L3midnonmotoraltX_vs_L1midnonmotor.dscalar.nii
comparison_L3midnonmotoraltX_vs_L2midnonmotor.dscalar.nii
comparison_L3midnonmotorIX_vs_L1midnonmotor.dscalar.nii
comparison_L3midnonmotorIX_vs_L2midnonmotor.dscalar.nii
comparison_R2midnonmotor_vs_R1midnonmotor.dscalar.nii
comparison_R2motor_vs_R1motor.dscalar.nii
comparison_R3highnonmotor_vs_R12highnonmotor.dscalar.nii
comparison_R3midnonmotoraltIX_vs_R1midnonmotor.dscalar.nii
comparison_R3midnonmotoraltIX_vs_R2midnonmotor.dscalar.nii
comparison_R3midnonmotorX_vs_R1midnonmotor.dscalar.nii
comparison_R3midnonmotorX_vs_R2midnonmotor.dscalar.nii


In [42]:
for x in files:
    if x.endswith('dscalar.nii'):
        print('calculating:    ' + x)
        zvalues = nib.load(x).get_data()
        pvals = scipy.stats.norm.sf(zvalues)
        pvals = pvals.T
        pvals_onlycortex = pvals[0:59411] ### This selects values only from the cortex
        pvals_onlycortex = pvals_onlycortex.T
        pvals_onlycortex_FDR = statsmodels.sandbox.stats.multicomp.multipletests(pvals_onlycortex[0], alpha=0.05, method='fdr_bh', is_sorted=False, returnsorted=False)
        ### Put cortical FDR corrected p values back to the original pvals matrix
        pvals_onlycortex_FDR[1].shape = (59411, 1)
        pvals[0:59411] = pvals_onlycortex_FDR[1]
        np.save(x + '_pvalues_onetailedcortexonly_FDR.npy', pvals.T[0])
        print('finished calculating:    ' + x)

calculating:    comparison_L2midnonmotor_vs_L1midnonmotor.dscalar.nii
finished calculating:    comparison_L2midnonmotor_vs_L1midnonmotor.dscalar.nii
calculating:    comparison_L2motor_vs_L1motor.dscalar.nii
finished calculating:    comparison_L2motor_vs_L1motor.dscalar.nii
calculating:    comparison_L3highnonmotor_vs_L12highnonmotor.dscalar.nii
finished calculating:    comparison_L3highnonmotor_vs_L12highnonmotor.dscalar.nii
calculating:    comparison_L3midnonmotoraltX_vs_L1midnonmotor.dscalar.nii
finished calculating:    comparison_L3midnonmotoraltX_vs_L1midnonmotor.dscalar.nii
calculating:    comparison_L3midnonmotoraltX_vs_L2midnonmotor.dscalar.nii
finished calculating:    comparison_L3midnonmotoraltX_vs_L2midnonmotor.dscalar.nii
calculating:    comparison_L3midnonmotorIX_vs_L1midnonmotor.dscalar.nii
finished calculating:    comparison_L3midnonmotorIX_vs_L1midnonmotor.dscalar.nii
calculating:    comparison_L3midnonmotorIX_vs_L2midnonmotor.dscalar.nii
finished calculating:    compari

In [54]:
### Transform to dscalar format
files = os.listdir(directory) #This is to update list of files in directory
for y in files:
    if y.endswith('.npy'):
        res = nib.load(directory2 + '/hcp.tmp.lh.dscalar.nii').get_data()
        cortL = np.squeeze(np.array(np.where(res != 0)[0], dtype=np.int32))
        res = nib.load(directory2 + '/hcp.tmp.rh.dscalar.nii').get_data()
        cortR = np.squeeze(np.array(np.where(res != 0)[0], dtype=np.int32))
        cortLen = len(cortL) + len(cortR)
        del res
        emb = np.load(y)
        emb.shape
        emb.shape = (91282, 1)
        tmp = nib.load('comparison_R1midcog_vs_R2midcog_peakfconn1003.dscalar.nii') #Just to take the shape; has to be dscalar with one map, and brain only
        tmp_cifti = nib.cifti2.load('comparison_R1midcog_vs_R2midcog_peakfconn1003.dscalar.nii')
        data = tmp_cifti.get_data() * 0
        mim = tmp.header.matrix[1]
        for idx, bm in enumerate(mim.brain_models):
            print ((idx, bm.index_offset, bm.brain_structure))
        img = nib.cifti2.Cifti2Image(emb.T, nib.cifti2.Cifti2Header(tmp.header.matrix))
        img.to_filename(y + '.dscalar.nii')

pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1


(0, 0, 'CIFTI_STRUCTURE_CORTEX_LEFT')
(1, 29696, 'CIFTI_STRUCTURE_CORTEX_RIGHT')
(2, 59412, 'CIFTI_STRUCTURE_ACCUMBENS_LEFT')
(3, 59547, 'CIFTI_STRUCTURE_ACCUMBENS_RIGHT')
(4, 59687, 'CIFTI_STRUCTURE_AMYGDALA_LEFT')
(5, 60002, 'CIFTI_STRUCTURE_AMYGDALA_RIGHT')
(6, 60334, 'CIFTI_STRUCTURE_BRAIN_STEM')
(7, 63806, 'CIFTI_STRUCTURE_CAUDATE_LEFT')
(8, 64534, 'CIFTI_STRUCTURE_CAUDATE_RIGHT')
(9, 65289, 'CIFTI_STRUCTURE_CEREBELLUM_LEFT')
(10, 73998, 'CIFTI_STRUCTURE_CEREBELLUM_RIGHT')
(11, 83142, 'CIFTI_STRUCTURE_DIENCEPHALON_VENTRAL_LEFT')
(12, 83848, 'CIFTI_STRUCTURE_DIENCEPHALON_VENTRAL_RIGHT')
(13, 84560, 'CIFTI_STRUCTURE_HIPPOCAMPUS_LEFT')
(14, 85324, 'CIFTI_STRUCTURE_HIPPOCAMPUS_RIGHT')
(15, 86119, 'CIFTI_STRUCTURE_PALLIDUM_LEFT')


pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1


(16, 86416, 'CIFTI_STRUCTURE_PALLIDUM_RIGHT')
(17, 86676, 'CIFTI_STRUCTURE_PUTAMEN_LEFT')
(18, 87736, 'CIFTI_STRUCTURE_PUTAMEN_RIGHT')
(19, 88746, 'CIFTI_STRUCTURE_THALAMUS_LEFT')
(20, 90034, 'CIFTI_STRUCTURE_THALAMUS_RIGHT')
(0, 0, 'CIFTI_STRUCTURE_CORTEX_LEFT')
(1, 29696, 'CIFTI_STRUCTURE_CORTEX_RIGHT')
(2, 59412, 'CIFTI_STRUCTURE_ACCUMBENS_LEFT')
(3, 59547, 'CIFTI_STRUCTURE_ACCUMBENS_RIGHT')
(4, 59687, 'CIFTI_STRUCTURE_AMYGDALA_LEFT')
(5, 60002, 'CIFTI_STRUCTURE_AMYGDALA_RIGHT')
(6, 60334, 'CIFTI_STRUCTURE_BRAIN_STEM')
(7, 63806, 'CIFTI_STRUCTURE_CAUDATE_LEFT')
(8, 64534, 'CIFTI_STRUCTURE_CAUDATE_RIGHT')
(9, 65289, 'CIFTI_STRUCTURE_CEREBELLUM_LEFT')
(10, 73998, 'CIFTI_STRUCTURE_CEREBELLUM_RIGHT')
(11, 83142, 'CIFTI_STRUCTURE_DIENCEPHALON_VENTRAL_LEFT')
(12, 83848, 'CIFTI_STRUCTURE_DIENCEPHALON_VENTRAL_RIGHT')
(13, 84560, 'CIFTI_STRUCTURE_HIPPOCAMPUS_LEFT')
(14, 85324, 'CIFTI_STRUCTURE_HIPPOCAMPUS_RIGHT')
(15, 86119, 'CIFTI_STRUCTURE_PALLIDUM_LEFT')

pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1


...

(17, 86676, 'CIFTI_STRUCTURE_PUTAMEN_LEFT')
(18, 87736, 'CIFTI_STRUCTURE_PUTAMEN_RIGHT')
(19, 88746, 'CIFTI_STRUCTURE_THALAMUS_LEFT')
(20, 90034, 'CIFTI_STRUCTURE_THALAMUS_RIGHT')
(0, 0, 'CIFTI_STRUCTURE_CORTEX_LEFT')
(1, 29696, 'CIFTI_STRUCTURE_CORTEX_RIGHT')
(2, 59412, 'CIFTI_STRUCTURE_ACCUMBENS_LEFT')
(3, 59547, 'CIFTI_STRUCTURE_ACCUMBENS_RIGHT')
(4, 59687, 'CIFTI_STRUCTURE_AMYGDALA_LEFT')
(5, 60002, 'CIFTI_STRUCTURE_AMYGDALA_RIGHT')
(6, 60334, 'CIFTI_STRUCTURE_BRAIN_STEM')
(7, 63806, 'CIFTI_STRUCTURE_CAUDATE_LEFT')
(8, 64534, 'CIFTI_STRUCTURE_CAUDATE_RIGHT')
(9, 65289, 'CIFTI_STRUCTURE_CEREBELLUM_LEFT')
(10, 73998, 'CIFTI_STRUCTURE_CEREBELLUM_RIGHT')
(11, 83142, 'CIFTI_STRUCTURE_DIENCEPHALON_VENTRAL_LEFT')

pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1



(12, 83848, 'CIFTI_STRUCTURE_DIENCEPHALON_VENTRAL_RIGHT')
(13, 84560, 'CIFTI_STRUCTURE_HIPPOCAMPUS_LEFT')
(14, 85324, 'CIFTI_STRUCTURE_HIPPOCAMPUS_RIGHT')
(15, 86119, 'CIFTI_STRUCTURE_PALLIDUM_LEFT')
(16, 86416, 'CIFTI_STRUCTURE_PALLIDUM_RIGHT')
(17, 86676, 'CIFTI_STRUCTURE_PUTAMEN_LEFT')
(18, 87736, 'CIFTI_STRUCTURE_PUTAMEN_RIGHT')
(19, 88746, 'CIFTI_STRUCTURE_THALAMUS_LEFT')
(20, 90034, 'CIFTI_STRUCTURE_THALAMUS_RIGHT')
(0, 0, 'CIFTI_STRUCTURE_CORTEX_LEFT')
(1, 29696, 'CIFTI_STRUCTURE_CORTEX_RIGHT')
(2, 59412, 'CIFTI_STRUCTURE_ACCUMBENS_LEFT')
(3, 59547, 'CIFTI_STRUCTURE_ACCUMBENS_RIGHT')
(4, 59687, 'CIFTI_STRUCTURE_AMYGDALA_LEFT')
(5, 60002, 'CIFTI_STRUCTURE_AMYGDALA_RIGHT')
(6, 60334, 'CIFTI_STRUCTURE_BRAIN_STEM')
(7, 63806, 'CIFTI_STRUCTURE_CAUDATE_LEFT')
(8, 64534, 'CIFTI_STRUCTURE_CAUDATE_RIGHT')
(9, 65289, 'CIFTI_STRUCTURE_CEREBELLUM_LEFT')
(10, 73998, 'CIFTI_STRUCTURE_CEREBELLUM_RIGHT')
(11, 83142, 'CIFTI_STRUCTURE_DIENCEPHALON_VENTRAL_LEFT')
(12, 83848, 'CIFTI_STRUCTURE_DIEN

pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1



(14, 85324, 'CIFTI_STRUCTURE_HIPPOCAMPUS_RIGHT')
(15, 86119, 'CIFTI_STRUCTURE_PALLIDUM_LEFT')
(16, 86416, 'CIFTI_STRUCTURE_PALLIDUM_RIGHT')
(17, 86676, 'CIFTI_STRUCTURE_PUTAMEN_LEFT')
(18, 87736, 'CIFTI_STRUCTURE_PUTAMEN_RIGHT')
(19, 88746, 'CIFTI_STRUCTURE_THALAMUS_LEFT')
(20, 90034, 'CIFTI_STRUCTURE_THALAMUS_RIGHT')
(0, 0, 'CIFTI_STRUCTURE_CORTEX_LEFT')
(1, 29696, 'CIFTI_STRUCTURE_CORTEX_RIGHT')
(2, 59412, 'CIFTI_STRUCTURE_ACCUMBENS_LEFT')
(3, 59547, 'CIFTI_STRUCTURE_ACCUMBENS_RIGHT')
(4, 59687, 'CIFTI_STRUCTURE_AMYGDALA_LEFT')
(5, 60002, 'CIFTI_STRUCTURE_AMYGDALA_RIGHT')
(6, 60334, 'CIFTI_STRUCTURE_BRAIN_STEM')
(7, 63806, 'CIFTI_STRUCTURE_CAUDATE_LEFT')
(8, 64534, 'CIFTI_STRUCTURE_CAUDATE_RIGHT')
(9, 65289, 'CIFTI_STRUCTURE_CEREBELLUM_LEFT')
(10, 73998, 'CIFTI_STRUCTURE_CEREBELLUM_RIGHT')
(11, 83142, 'CIFTI_STRUCTURE_DIENCEPHALON_VENTRAL_LEFT')
(12, 83848, 'CIFTI_STRUCTURE_DIENCEPHALON_VENTRAL_RIGHT')
(13, 84560, 'CIFTI_STRUCTURE_HIPPOCAMPUS_LEFT')
(14, 85324, 'CIFTI_STRUCTURE_HIPP

In [ ]:
### 5) Open corrected maps with wb_view and use a threshold of 0.05